# The paleopiezometry module

**What is paleopizometry?**  
Paleopiezometers refer to microstructural features of deformed rocks that vary with the magnitude of the applied differential stress under which they formed (Twiss and Moores, 2006). They serve to infer differential stresses in the geological past, hence the name paleopiezometry, and they are an essential tool for validating rheological models of the lithosphere. The most prevalent microstructure used for this purpose is the average recrystallized (apparent) grain size. This choice is due to its ease of measurement in recrystallized rocks.

> 📣 The GrainSizeTools script includes a **Jupyter notebook template** to promote the reproducibility of palaeopizometry studies. Use it as a template by deleting and adding as necessary and use it as supplementary material to your study so that anyone can reproduce your results.

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import grain_size_tools as gst

## Estimate the stress using the average grain size

### Before you begin

The paleopiezometry module includes a function named ``calc_diffstress()`` for estimating differential stress based on average recrystallized grain sizes. The function requires careful consideration of the following factors for correct differential stress estimation:

1. definition of the mineral phase and the piezometer relationship to be used
2. enter the (apparent) grain size as **equivalent circular diameters** in microns
3. enter a specific average (arithmetic mean, median, etc.) with **no stereological correction**,
4. set the stress framework, either uniaxial compression/extension or plane stress, for proper stress correction.

Regarding the first factor, GrainSizeTools includes a database of paleopiezometric relations, including the most common mineral phases quartz, calcite, olivine and albite feldspar. In addition, the script facilitates the writing of ad hoc piezometric relations. As we will see later, it is also very easy to add new piezometric relationships.

For the second factor, the function assumes by default that the average grain size you specify has been estimated using equivalent circular diameters (ECDs). As some piezometric relationships, especially those established longer ago, were established using linear intercepts (LI) rather than ECDs, the function will automatically convert ECDs to linear intercepts if necessary using the Hoff and Rhines (1968) correction. This means that **you do not need to worry about whether the piezometer was originally calibrated using linear intercepts or not**, always use averages based on equivalent circular diameters in microns. The function will explicitly warn you when using this ECD ti LI of conversion.

The third factor is key for the correct estimation of differential stress, as each paleopiezometry relationship has been calibrated to a specific average grain size (e.g. the arithmetic mean, median or RMS mean) and thus **only gives valid results if the same type of average is used**. In addition, **you must not apply any type of stereological correction to the calculated grain average**, if the author(s) of the piezometer used any type of stereological correction during calibration, your input will be automatically corrected by the function.

The fourth factor means that the user must decide whether or not to correct the differential stress estimate for plane strain using the correction factor suggested by Paterson and Olgaard (2000). The reason for this is that piezometer calibration experiments are mainly performed in uniaxial compression, whereas natural shear zones behave approximately like plane-strain volumes.

### Use of the piezometric database

In [35]:
# Load the piezometric database
version, metadata, db = gst.piezometers.load_piezometers_from_yaml(None)
print(f"Piezometric database v{version} loaded")

Piezometric database v2024.06.06 loaded


You can list all available piezometric relations using `gst.piezometers.list_piezometers(db)`. For example:

In [36]:
gst.piezometers.list_piezometers(db)

quartz:
   Stipp_Tullis
   Stipp_Tullis_BLG
   Holyoke
   Holyoke_BLG
   Cross
   Cross_HR
   Shimizu
   Twiss
olivine:
   VanderWal_wet
   Jung_Karato
calcite:
   Rutter_SGR
   Rutter_GBM
   Barnhoorn
   Platt_Bresser
   Valcke
feldspar:
   Post_Tullis_BLG


If you want to get the details of a particular piezometric relationship simply pass the name of the relationship inside the parenthesis. Remember that the relationship between recrystallized grain size and differential stress is

$$\sigma_d = B\ d^{-m}$$

where $\sigma_d$ and $d$ are the differential stress and the average grain size respectively.

In [37]:
gst.piezometers.summary(db.quartz.__dict__['Twiss'])

year: 1977
reference: https://www.doi.org/10.1007/BF01637105
B: 550.0
m: 0.68
warn: Please ensure that you have entered the apparent grain size as the arithmetic mean grain size.
linear_intercepts: False
correction_factor: 1.5
notes: Twiss (1977) piezometer was calibrated using the linear intercepts (LI) multiplied
by 1.5 (correction factor). Equivalent circular diameters (ECD) without stereological
correction must be converted to LIs using the empirical equation of De Hoff and
Rhines (1968) as follows LI = (1.5 / sqrt(4/pi)) * ECD



The output shows all properties stored in the database for that piezometric relationship, including the constants *B* and *m*, the type of average expected, whether the relationship was calibrated using linear intercepts, and any relevant warnings.

### Estimating differential stresses

Let us first look at the documentation of `calc_diffstress`:

In [38]:
gst.piezometers.calc_diffstress?

Signature: gst.piezometers.calc_diffstress(piezometer, grain_size, correction=False)
Docstring:
Apply different piezometric relation to estimate differential stress
based on average apparent grain sizes. The piezometric relation has
the following general form:

diff_stress = B * grain_size**-m

where differential stress is in [MPa], B is an experimentally
derived parameter in [MPa micron**m], grain_size is the aparent grain
size in [microns], and m is an experimentally derived exponent.

Parameters
----------
piezometer : SimpleNamespace
    the piezometric relation

grain_size : positive scalar or array-like
    the apparent grain size in microns

correction : bool, default False
    correct the stress values from uniaxial compression (experiments) 
    to plane stress (nature) using the Paterson and Olgaard (2000)
    approach; i.e. multiplying the paleopiezometer by 2/sqrt(3)

References
-----------
Paterson and Olgaard (2000) https://doi.org/10.1016/S0191-8141(00)00042-0
de Hoff an

As indicated in the documentation, `calc_diffstress()` requires two inputs: (1) the piezometer selected from the database and (2) the average grain size in microns. A few examples are given below:

In [39]:
gst.piezometers.calc_diffstress(db.quartz.__dict__['Twiss'], grain_size=12)

Calculated differential stress = 101.51 MPa

INFO:
Please ensure that you have entered the apparent grain size as the arithmetic mean grain size.


The function returns the calculated differential stress (in MPa) plus some relevant information about the corrections made and the type of average expected as input.

As most piezometric calibrations have been calibrated using uniaxial compression deformation experiments, let's correct this estimate for planar stress using the correction suggested by Paterson and Olgaard (2000). This is done by passing a new parameter within the function as follows (note the slightly different value of differential stress):

In [40]:
gst.piezometers.calc_diffstress(db.quartz.__dict__['Twiss'], grain_size=12, correction=True)

Calculated differential stress = 117.22 MPa

INFO:
Please ensure that you have entered the apparent grain size as the arithmetic mean grain size.
The differential stress was corrected for plane stress using the methodology outlined in Paterson and Olgaard (2000).


## How to synthesise a set of piezometric values

TODO

> ⚠️ **Never combine averages -or any other statistic- as if they were data points**. In the context of meta-analysis, which is a statistical procedure for combining data from multiple studies or experiments, a common mistake is to take different estimates and averaged them. However, the stress estimates are already an average and calculating a mean of the averages is mathematically incorrect because this approach ignores the uncertainty of the average. Similarly, to look at the raw grain size data from all the grain maps and make the estimate from there is also incorrect because this ignores the uncertainty within and between experimental conditions. The correct procedure here is to take the weighted mean of the averages, where each mean is weighted by its variance or the squared standard error of the mean (SEM). The GrainSize Tools script has a function to do this named ```weighted_mean_and_se()```

## Summary of piezometric relationships in the database

$$\sigma_d = B\ d^{-m}$$
$$d = A\ \sigma_d^{-p}$$

$B$ and $m$ relate to $A$ and $p$ as follows:
 
$$B = A^{1/p}$$
$$m = 1/p$$

$A$ and $B$ are in $\mu m\ MPa^{p,m}$

### Quartz piezometric relationships

|          calibration           |       reference        |    DRX    |        average         |    A    |  p   |   B    |  m   |
| :----------------------------: | :--------------------: | :-------: | :--------------------: | :-----: | :--: | :----: | :--: |
|     Cross et al. (2017)*§*     |      ``'Cross'``       | BLG, SGR  |        RMS mean        | 8128.3  | 1.41 | 593.0  | 0.71 |
|     Cross et al. (2017)*§*     |     ``'Cross_hr'``     | BLG, SGR  |        RMS mean        | 16595.9 | 1.59 | 450.9  | 0.63 |
| Holyoke & Kronenberg (2010)*¶* |     ``'Holyoke'``      | SGR + GBM |        RMS mean        |  2451   | 1.26 | 490.3  | 0.79 |
| Holyoke & Kronenberg (2010)*¶* |   ``'Holyoke_BLG'``    |    BLG    |        RMS mean        |   39    | 0.54 | 883.9  | 1.85 |
|        Shimizu (2008)‡         |     ``'Shimizu'``      | SGR + GBM | Median in log(e) scale |  1525   | 1.25 |  352   | 0.8  |
|    Stipp and Tullis (2003)     |   ``'Stipp_Tullis'``   | SGR + GBM |        RMS mean        | 3630.8  | 1.26 | 669.0  | 0.79 |
|    Stipp and Tullis (2003)     | ``'Stipp_Tullis_BLG'`` |    BLG    |        RMS mean        |   78    | 0.61 | 1264.1 | 1.64 |
|         Twiss (1977)†          |      ``'Twiss'``       | SGR + GBM |      arith. mean       |  1230   | 1.47 |  550   | 0.68 |

*† Apparent grain size measured as equivalent circular diameters (ECD) with no stereological correction and reported in microns. The use of non-linear scales is indicated*    
*‡ Shimizu piezometer requires to provide the temperature during deformation in K*    
*§ Cross et al. (2017) reanalysed the samples of Stipp and Tullis (2003) using EBSD data for reconstructing the grains. Specifically, they use grain maps with a 1 m and a 200 nm (hr - high-resolution) step sizes . This is the preferred piezometer for quartz when grain size data comes from EBSD maps*  
*¶ Holyoke and Kronenberg (2010) provides a recalibration of the Stipp and Tullis (2003) piezometer*  

> To check:  
> Tockle and Hirth 2021 https://doi.org/10.1029/2020JB021475  
> Bishop (1996)  
> Kidder et al. (2016) https://doi.org/10.1016/j.jsg.2015.12.004  
> Heilbronner & Kilian (2017)  
> Richter et al. (2018)  
> Soleymani et al. (2020) https://doi.org/10.1130/G46972.1

### Olivine piezometric relationships

|        calibration         |      reference      | DRX  | dry/wet |   average   |  A    |  p   |    B    |  m   |
| :------------------------: | :-----------------: | :--: | :-----: | :---------: | :---: | :--: | :-----: | :--: |
|  Jung and Karato (2001)§   |  ``'Jung_Karato'``  | BLG  |   wet   | arith. mean | 25704 | 1.18 | 5461.03 | 0.85 |
| Van der Wal et al. (1993)§ | ``'VanderWal_wet'`` |      | dry/wet | arith. mean | 15000 | 1.33 | 1355.4  | 0.75 |
|    Tasaka et al. (2015)    |  ``'Tasaka_wet'``   |      |   wet   | arith. mean | 6310  | 1.33 |  719.7  | 0.75 |

*§ These piezometers were originally calibrated using linear intercepts (LI) instead of ECD*  

> To check: Add Twiss 1977, Karato 1980 and Ross et al 1980?


### Calcite piezometric relationships

|        calibration          |      reference      |   DRX    |   average   |   A    |  p   |    B    |  m   |
| :-------------------------: | :-----------------: | :------: | :---------: | :----: | :--: | :-----: | :--: |
|   Barnhoorn et al. (2004)   |   ``'Barnhoorn'``   | SRG, GBM | arith. mean | 2134.4 | 1.22 | 537.03  | 0.82 |
| Platt and De Bresser (2017) | ``'Platt_Bresser'`` | BLG, SGR |  RMS mean   |  2141  | 1.22 | 538.40  | 0.82 |
|        Rutter (1995)        |  ``'Rutter_SGR'``   |   SGR    | arith. mean | 2026.8 | 1.14 | 812.83  | 0.88 |
|        Rutter (1995)        |  ``'Rutter_GBM'``   |   GBM    | arith. mean | 7143.8 | 1.12 | 2691.53 | 0.89 |
|    Valcke et al. (2015)     |    ``'Valcke'``     | BLG, SGR | arith. mean | 79.43  | 0.6  | 1467.92 | 1.67 |  

> To check:: Add Twiss 1977 and Schmidt et al. 1980?

### Feldspar piezometric relationships

|       calibration       |       reference       | DRX  | average |  A   |  p   |   B   |  m   |
| :---------------------: | :-------------------: | :--: | :-----: | :--: | :--: | :---: | :--: |
| Post and Tullis (1999)§ | ``'Post_Tullis_BLG'`` | BLG  | Median  |  55  | 0.66 | 433.4 | 1.52 |

*§ These piezometers were originally calibrated using linear intercepts (LI) instead of ECD*

> TODO:  
> Add Speciale et al. (2022) https://doi.org/10.1016/j.jsg.2021.104495


### Orthopyroxene piezometric relationships

> TODO:  
> Linckens et al. 2014 https://doi.org/10.1016/j.epsl.2013.11.037  
> Brujin ans Skemer 2014 https://doi.org/10.1002/2014GL060607  
> Speciale et al. 2022 https://doi.org/10.1016/j.jsg.2021.104495


### Halite piezometric relationships

> TODO:  
> Guillopé and Poirier 1979, https://doi.org/10.1029/JB084iB10p05557  
> Ter Heege et al. 2005 https://doi.org/10.1016/j.tecto.2004.10.002


In [41]:
import sys
from datetime import date    
today = date.today().isoformat()
import matplotlib as mpl

print(f'Notebook tested in {today} using:')
print('Python', sys.version)
print('Numpy', np.__version__)
print('Matplotlib', mpl.__version__)

Notebook tested in 2026-02-28 using:
Python 3.11.14 (main, Feb 12 2026, 00:37:49) [MSC v.1944 64 bit (AMD64)]
Numpy 2.4.2
Matplotlib 3.10.8
